In [9]:
!export REPLICATE_API_TOKEN=r8_OowioSTEHuawUOZ01mRiYqBKVmT3dv14Bnnfa

In [10]:
# set os varibale 
import os
os.environ['REPLICATE_API_TOKEN'] ="r8_OowioSTEHuawUOZ01mRiYqBKVmT3dv14Bnnfa"

In [11]:
import replicate



In [12]:
output = replicate.run(
    "jagilley/controlnet-hough:854e8727697a057c525cdb45ab037f64ecca770a1769cc52287c2e56472a247b",
    input={
        "eta": 0,
        "image": "https://replicate.delivery/pbxt/IJZOELWrncBcjdE1s5Ko8ou35ZOxjNxDqMf0BhoRUAtv76u4/room.png",
        "scale": 9,
        "prompt": "a cheerful modernist bedroom",
        "a_prompt": "best quality, extremely detailed",
        "n_prompt": "longbody, lowres, bad anatomy, bad hands, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality",
        "ddim_steps": 20,
        "num_samples": "1",
        "value_threshold": 0.1,
        "image_resolution": "512",
        "detect_resolution": 512,
        "distance_threshold": 0.1
    }
)
print(output)

ReplicateError: You did not pass an authentication token

In [14]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import os

# Download YOLOv3 weights and configuration file
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://github.com/pjreddie/darknet/blob/master/cfg/yolov3.cfg?raw=true -O yolov3.cfg
!wget https://github.com/pjreddie/darknet/blob/master/data/coco.names?raw=true -O coco.names

# Load COCO class labels
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f]

def object_detection(image_path, furniture_classes=["bed", "sofa", "diningtable"]):
    # Use PIL to read the image
    img = Image.open(image_path)
    img = np.array(img)  # Convert PIL Image to NumPy array
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            # Get the class label from the COCO class labels file (coco.names)
            label = classes[class_id]

            # Only keep objects related to furniture
            if label.lower() in furniture_classes and confidence > 0.5:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    return boxes, confidences, class_ids

# Load YOLOv3 model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Example usage:
input_image_path = "/content/room_image.jpg"  # Replace with the correct file path
boxes, _, _ = object_detection(input_image_path)

# Visualize bounding boxes on the input image
image = cv2.imread(input_image_path)
for box in boxes:
    x, y, w, h = box
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

cv2.imwrite("output_image.jpg", image)



!mkdir extracted_boxes



def extract_labeled_boxes(image_path, boxes, class_ids, classes, output_folder="extracted_boxes"):
    img = cv2.imread(image_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for i, (box, class_id) in enumerate(zip(boxes, class_ids)):
        x, y, w, h = box
        object_image = img[y:y + h, x:x + w]
        label = classes[class_id]

        # Save the image with the corresponding label
        cv2.imwrite(f"{output_folder}/{label}object{i}.jpg", object_image)

def object_detection_with_visualization(image_path, output_folder="output_images"):
    img = Image.open(image_path)
    img_np = np.array(img)
    height, width, _ = img_np.shape

    blob = cv2.dnn.blobFromImage(img_np, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.5:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Draw bounding boxes and labels on the image
    for i, box in enumerate(boxes):
        x, y, w, h = box
        label = classes[class_ids[i]]
        cv2.rectangle(img_np, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(img_np, f"{label} {confidences[i]:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Save the image with bounding boxes and labels
    output_path = os.path.join(output_folder, "output_image.jpg")
    cv2.imwrite(output_path, cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR))
    print(f"Visualized image saved at: {output_path}")

    return boxes, confidences, class_ids

# Example usage:
input_image_path = "/content/room_image.jpg"  # Replace with the correct file path
boxes, _, class_ids = object_detection_with_visualization(input_image_path)

# Extract labeled objects from the image
extract_labeled_boxes(input_image_path, boxes, class_ids, classes)

ModuleNotFoundError: No module named 'cv2'